In [1]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_context("talk")
sns.set_style("darkgrid")

In [2]:
import itertools 

def create_df_from_combinations(input_dict):
    variable_names = []
    product_input_tuple = ()
    for key, value in input_dict.items():
        variable_names.append(key)
        product_input_tuple += (value,) # this is one way to add things to a tuple, apparently

    return pd.DataFrame(list(itertools.product(*product_input_tuple)), columns=variable_names)

Scenarios:
- Archetypes
- Drugs: DP vs 14-day drug (archetype dependent)
- Screening type: IPT vs IST
- Interval: once per term vs once per month
- Background transmission intensity: ~4 levels
- Out of school rates: 0,15%,30% (combinatorial)
- Within-school coverage: default vs all (not combinatorial)
- Treatment-seeking rate: low vs default (not combinatorial)
- Bednet coverage: default vs high (not combinatorial)
- Extreme best case scenario (not combinatorial)

For "not combinatorial", this means adding a single row for that variation, rather than including all combinations


- Sahel: SPAQ (would also consider ASAQ as an alternative)
- Coastal West Africa: SPAQ (would also consider ASAQ as an alternative)
- Central Africa: ASAQ
- East Africa: ASAQ and SPAQ (the later is only because Sarah mentioned they are trying it for SMC in some areas of Uganda)
- Southern Africa: ASAQ"

In [9]:
def _add_dict_to_df_list(df_list, d):
    new_df = pd.DataFrame(d, index=[0])
    df_list.append(new_df)

In [10]:
def basic_scenarios_southern():
    drugs = ["ASAQ","DP"]
    
    scenario_variations = {
        "drug_type": drugs,
        "screen_type": ["IPT", "IST"],
        "interval": ["term", "month"],
        "itn_coverage": ["default"],
        "hs_rate": ["default"],
        "within_school_coverage": [0.9],
        "out_of_school_rate": [0,0.15,0.3],
        "target_age_range": ["default"],
        "smc_on": [False]
    }

    df_combinatorial = create_df_from_combinations(scenario_variations)

    df_list = []
    
    # Default comparator is DP IPT, once per term, with 15% out of school, and default ITN/HS.
    
    # Higher ITN cov
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "term",
        "itn_coverage": "high",
        "hs_rate": "default",
        "within_school_coverage": 0.9,
        "out_of_school_rate": 0.15,
        "target_age_range": "default",
        "smc_on": False
    })
    
    # Lower HS rate
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "term",
        "itn_coverage": "default",
        "hs_rate": "low",
        "within_school_coverage": 0.9,
        "out_of_school_rate": 0.15,
        "target_age_range": "default",
        "smc_on": False
    })
    
    # 100% within school coverage
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "term",
        "itn_coverage": "default",
        "hs_rate": "default",
        "within_school_coverage": 1.0,
        "out_of_school_rate": 0.15,
        "target_age_range": "default",
        "smc_on": False
    })
    
    # Extreme optimism - daily DP for all school-age kids
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "day",
        "itn_coverage": "default",
        "hs_rate": "default",
        "within_school_coverage": 1.0,
        "out_of_school_rate": 0,
        "target_age_range": "default",
        "smc_on": False
    })
    
    # Extreme optimism, v2 - daily DP for all u5 kids
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "day",
        "itn_coverage": "default",
        "hs_rate": "default",
        "within_school_coverage": 1.0,
        "out_of_school_rate": 0,
        "target_age_range": "u5",
        "smc_on": False
    })
    
    # Extreme optimism, v3 - daily DP for all u16 kids
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "day",
        "itn_coverage": "default",
        "hs_rate": "default",
        "within_school_coverage": 1.0,
        "out_of_school_rate": 0,
        "target_age_range": "u16",
        "smc_on": False
    })
    
    # Baseline: ITN/HS but no IPT
    _add_dict_to_df_list(df_list, {
        "drug_type": "None",
        "screen_type": "None",
        "interval": "None",
        "itn_coverage": "default",
        "hs_rate": "default",
        "within_school_coverage": 0.9,
        "out_of_school_rate": 0,
        "target_age_range": "default",
        "smc_on": False
    })
    
    # Comparator: No IPT, increased ITN coverage.
    _add_dict_to_df_list(df_list, {
        "drug_type": "None",
        "screen_type": "None",
        "interval": "None",
        "itn_coverage": "high",
        "hs_rate": "default",
        "within_school_coverage": 0.9,
        "out_of_school_rate": 0.15,
        "target_age_range": "default",
        "smc_on": False
    })
    
    # Comparator: No school-age IPT, but instead give 90% of under-5s drugs with same coverage and timing
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "term",
        "itn_coverage": "default",
        "hs_rate": "default",
        "within_school_coverage": 0.9,
        "out_of_school_rate": 0,
        "target_age_range": "u5",
        "smc_on": False
    })

    # Comparator to above: Give approximately same number of doses to school-age kids, with same timing
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "term",
        "itn_coverage": "default",
        "hs_rate": "default",
        "within_school_coverage": 0.63,
        "out_of_school_rate": 0,
        "target_age_range": "default",
        "smc_on": False
    })
    
    # Ivermectin:
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "term",
        "itn_coverage": "default",
        "hs_rate": "default",
        "within_school_coverage": 0.9,
        "out_of_school_rate": 0.15,
        "target_age_range": "default",
        "smc_on": False,
        "ivermectin": True,
        "primaquine": False
    })
    
    # Primaquine:
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "term",
        "itn_coverage": "default",
        "hs_rate": "default",
        "within_school_coverage": 0.9,
        "out_of_school_rate": 0.15,
        "target_age_range": "default",
        "smc_on": False,
        "ivermectin": False,
        "primaquine": True
    })
    
    # Ivermectin and Primaquine:
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "term",
        "itn_coverage": "default",
        "hs_rate": "default",
        "within_school_coverage": 0.9,
        "out_of_school_rate": 0.15,
        "target_age_range": "default",
        "smc_on": False,
        "ivermectin": True,
        "primaquine": True
    })
    
    # Extreme optimism, v4 - daily DP AND primaquine/ivermectin for all SACs
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "day",
        "itn_coverage": "default",
        "hs_rate": "default",
        "within_school_coverage": 1.0,
        "out_of_school_rate": 0,
        "target_age_range": "default",
        "smc_on": False,
        "ivermectin": True,
        "primaquine": True
    })
    
    df_not_combinatorial = pd.concat(df_list, ignore_index=True)
    
    scenario_df = pd.concat([df_combinatorial, df_not_combinatorial], ignore_index=True)
    scenario_df.fillna({"smc_on": False,
                        "ivermectin": False,
                        "primaquine": False}, inplace=True)
        
    scenario_df["scenario_number"] = range(len(scenario_df))
    scenario_df["archetype"] = "Southern"
    
    return scenario_df

In [11]:
def basic_scenarios_sahel():
    drugs = ["SPAQ","DP"]
    
    scenario_variations = {
        "drug_type": drugs,
        "screen_type": ["IPT", "IST"],
        "interval": ["term", "month"],
        "itn_coverage": ["default"],
        "hs_rate": ["default"],
        "within_school_coverage": [0.9],
        "out_of_school_rate": [0,0.15,0.3],
        "target_age_range": ["default"],
        "smc_on": [True]
    }

    df_combinatorial = create_df_from_combinations(scenario_variations)

    df_list = []
    
    # Default comparator is DP IPT, once per term, with 15% out of school, and default ITN/HS.
    
    # Higher ITN cov
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "term",
        "itn_coverage": "high",
        "hs_rate": "default",
        "within_school_coverage": 0.9,
        "out_of_school_rate": 0.15,
        "target_age_range": "default",
        "smc_on": True
    })
    
    # Lower HS rate
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "term",
        "itn_coverage": "default",
        "hs_rate": "low",
        "within_school_coverage": 0.9,
        "out_of_school_rate": 0.15,
        "target_age_range": "default",
        "smc_on": True
    })
    
    # 100% within school coverage
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "term",
        "itn_coverage": "default",
        "hs_rate": "default",
        "within_school_coverage": 1.0,
        "out_of_school_rate": 0.15,
        "target_age_range": "default",
        "smc_on": True
    })
    
    # Extreme optimism - daily DP for all school-age kids
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "day",
        "itn_coverage": "default",
        "hs_rate": "default",
        "within_school_coverage": 1.0,
        "out_of_school_rate": 0,
        "target_age_range": "default",
        "smc_on": True
    })
    
    # Extreme optimism, v2 - daily DP for all u5 kids
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "day",
        "itn_coverage": "default",
        "hs_rate": "default",
        "within_school_coverage": 1.0,
        "out_of_school_rate": 0,
        "target_age_range": "u5",
        "smc_on": True
    })
    
    # Extreme optimism, v3 - daily DP for all u16 kids
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "day",
        "itn_coverage": "default",
        "hs_rate": "default",
        "within_school_coverage": 1.0,
        "out_of_school_rate": 0,
        "target_age_range": "u16",
        "smc_on": True
    })
    
    # Baseline: ITN/HS/SMC but no IPT
    _add_dict_to_df_list(df_list, {
        "drug_type": "None",
        "screen_type": "None",
        "interval": "None",
        "itn_coverage": "default",
        "hs_rate": "default",
        "within_school_coverage": 0.9,
        "out_of_school_rate": 0,
        "target_age_range": "default",
        "smc_on": True
    })
    
    # Comparator: No IPT, increased ITN coverage.
    _add_dict_to_df_list(df_list, {
        "drug_type": "None",
        "screen_type": "None",
        "interval": "None",
        "itn_coverage": "high",
        "hs_rate": "default",
        "within_school_coverage": 0.9,
        "out_of_school_rate": 0.15,
        "target_age_range": "default",
        "smc_on": True
    })
    
    # Comparator: No school-age IPT, but instead give 90% of under-5s drugs with same coverage and timing
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "term",
        "itn_coverage": "default",
        "hs_rate": "default",
        "within_school_coverage": 0.9,
        "out_of_school_rate": 0,
        "target_age_range": "u5",
        "smc_on": True
    })

    # Comparator to above: Give approximately same number of doses to school-age kids, with same timing
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "term",
        "itn_coverage": "default",
        "hs_rate": "default",
        "within_school_coverage": 0.63,
        "out_of_school_rate": 0,
        "target_age_range": "default",
        "smc_on": True
    })
    
    # Baseline but without SMC
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "term",
        "itn_coverage": "default",
        "hs_rate": "default",
        "within_school_coverage": 0.9,
        "out_of_school_rate": 0.15,
        "target_age_range": "default",
        "smc_on": False
    })
    
    # Comparator: SMC extended to under-10s
    _add_dict_to_df_list(df_list, {
        "drug_type": "None",
        "screen_type": "None",
        "interval": "None",
        "itn_coverage": "default",
        "hs_rate": "default",
        "within_school_coverage": 0.9,
        "out_of_school_rate": 0,
        "target_age_range": "default",
        "smc_on": True,
        "smc_age_range": "u10"
    })
    
    # Comparator: SMC extended to under-10s
    _add_dict_to_df_list(df_list, {
        "drug_type": "None",
        "screen_type": "None",
        "interval": "None",
        "itn_coverage": "default",
        "hs_rate": "default",
        "within_school_coverage": 0.9,
        "out_of_school_rate": 0,
        "target_age_range": "default",
        "smc_on": True,
        "smc_age_range": "u15"
    })
    
    # Extreme optimism, v4 - daily DP AND primaquine/ivermectin for all SACs
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "day",
        "itn_coverage": "default",
        "hs_rate": "default",
        "within_school_coverage": 1.0,
        "out_of_school_rate": 0,
        "target_age_range": "default",
        "ivermectin": True,
        "primaquine": True,
        "smc_on": True
    })
    
    
    df_not_combinatorial = pd.concat(df_list, ignore_index=True)
    
    # Mini combinatorial to look at primaquine/ivermectin variations
    scenario_variations = {
        "drug_type": ["DP"],
        "screen_type": ["IPT"],
        "interval": ["term"],
        "itn_coverage": ["default"],
        "hs_rate": ["default"],
        "within_school_coverage": [0.9],
        "out_of_school_rate": [0.15],
        "target_age_range": ["default"],
        "smc_on": [False, True],
        "ivermectin": [False, True],
        "primaquine": [False, True]
    }

    df_combinatorial_iver_prim = create_df_from_combinations(scenario_variations)
    
    
    scenario_df = pd.concat([df_combinatorial, df_not_combinatorial, df_combinatorial_iver_prim], ignore_index=True)
    scenario_df.fillna({"smc_on": False,
                        "smc_age_range": "default",
                        "ivermectin": False,
                        "primaquine": False}, inplace=True)
    
    scenario_df["scenario_number"] = range(len(scenario_df))
    scenario_df["archetype"] = "Sahel"
    
    return scenario_df

In [12]:
def basic_scenarios_central():
    drugs = ["ASAQ","DP"]
    
    scenario_variations = {
        "drug_type": drugs,
        "screen_type": ["IPT", "IST"],
        "interval": ["term", "month"],
        "itn_coverage": ["default"],
        "hs_rate": ["default"],
        "within_school_coverage": [0.9],
        "out_of_school_rate": [0,0.15,0.3],
        "target_age_range": ["default"],
        "smc_on": [False]
    }

    df_combinatorial = create_df_from_combinations(scenario_variations)

    df_list = []
    
    # Default comparator is DP IPT, once per term, with 15% out of school, and default ITN/HS.
    
    # Higher ITN cov
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "term",
        "itn_coverage": "high",
        "hs_rate": "default",
        "within_school_coverage": 0.9,
        "out_of_school_rate": 0.15,
        "target_age_range": "default",
        "smc_on": False
    })
    
    # Lower HS rate
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "term",
        "itn_coverage": "default",
        "hs_rate": "low",
        "within_school_coverage": 0.9,
        "out_of_school_rate": 0.15,
        "target_age_range": "default",
        "smc_on": False
    })
    
    # 100% within school coverage
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "term",
        "itn_coverage": "default",
        "hs_rate": "default",
        "within_school_coverage": 1.0,
        "out_of_school_rate": 0.15,
        "target_age_range": "default",
        "smc_on": False
    })
    
    # Extreme optimism - daily DP for all school-age kids
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "day",
        "itn_coverage": "default",
        "hs_rate": "default",
        "within_school_coverage": 1.0,
        "out_of_school_rate": 0,
        "target_age_range": "default",
        "smc_on": False
    })
    
    # Extreme optimism, v2 - daily DP for all u5 kids
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "day",
        "itn_coverage": "default",
        "hs_rate": "default",
        "within_school_coverage": 1.0,
        "out_of_school_rate": 0,
        "target_age_range": "u5",
        "smc_on": False
    })
    
    # Extreme optimism, v3 - daily DP for all u16 kids
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "day",
        "itn_coverage": "default",
        "hs_rate": "default",
        "within_school_coverage": 1.0,
        "out_of_school_rate": 0,
        "target_age_range": "u16",
        "smc_on": False
    })
    
    # Baseline: ITN/HS but no IPT
    _add_dict_to_df_list(df_list, {
        "drug_type": "None",
        "screen_type": "None",
        "interval": "None",
        "itn_coverage": "default",
        "hs_rate": "default",
        "within_school_coverage": 0.9,
        "out_of_school_rate": 0,
        "target_age_range": "default",
        "smc_on": False
    })
    
    # Comparator: No IPT, increased ITN coverage.
    _add_dict_to_df_list(df_list, {
        "drug_type": "None",
        "screen_type": "None",
        "interval": "None",
        "itn_coverage": "high",
        "hs_rate": "default",
        "within_school_coverage": 0.9,
        "out_of_school_rate": 0.15,
        "target_age_range": "default",
        "smc_on": False
    })
    
    # Comparator: No school-age IPT, but instead give 90% of under-5s drugs with same coverage and timing
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "term",
        "itn_coverage": "default",
        "hs_rate": "default",
        "within_school_coverage": 0.9,
        "out_of_school_rate": 0,
        "target_age_range": "u5",
        "smc_on": False
    })

    # Comparator to above: Give approximately same number of doses to school-age kids, with same timing
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "term",
        "itn_coverage": "default",
        "hs_rate": "default",
        "within_school_coverage": 0.63,
        "out_of_school_rate": 0,
        "target_age_range": "default",
        "smc_on": False
    })
    
    # Ivermectin:
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "term",
        "itn_coverage": "default",
        "hs_rate": "default",
        "within_school_coverage": 0.9,
        "out_of_school_rate": 0.15,
        "target_age_range": "default",
        "smc_on": False,
        "ivermectin": True,
        "primaquine": False
    })
    
    # Primaquine:
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "term",
        "itn_coverage": "default",
        "hs_rate": "default",
        "within_school_coverage": 0.9,
        "out_of_school_rate": 0.15,
        "target_age_range": "default",
        "smc_on": False,
        "ivermectin": False,
        "primaquine": True
    })
    
    # Ivermectin and Primaquine:
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "term",
        "itn_coverage": "default",
        "hs_rate": "default",
        "within_school_coverage": 0.9,
        "out_of_school_rate": 0.15,
        "target_age_range": "default",
        "smc_on": False,
        "ivermectin": True,
        "primaquine": True
    })
    
    # Extreme optimism, v4 - daily DP AND primaquine/ivermectin for all SACs
    _add_dict_to_df_list(df_list, {
        "drug_type": "DP",
        "screen_type": "IPT",
        "interval": "day",
        "itn_coverage": "default",
        "hs_rate": "default",
        "within_school_coverage": 1.0,
        "out_of_school_rate": 0,
        "target_age_range": "default",
        "smc_on": False,
        "ivermectin": True,
        "primaquine": True
    })
    
    df_not_combinatorial = pd.concat(df_list, ignore_index=True)
    
    scenario_df = pd.concat([df_combinatorial, df_not_combinatorial], ignore_index=True)
    scenario_df.fillna({"smc_on": False,
                        "ivermectin": False,
                        "primaquine": False}, inplace=True)
        
    scenario_df["scenario_number"] = range(len(scenario_df))
    scenario_df["archetype"] = "Central"
    
    return scenario_df

In [13]:
def basic_scenarios(archetype):
    if archetype == "Southern":
        return basic_scenarios_southern()
    elif archetype == "Sahel":
        return basic_scenarios_sahel()
    elif archetype == "Central":
        return basic_scenarios_central()
    else:
        return NotImplementedError

In [14]:
# archetype_list = ["Southern", "Central", "Eastern", "Coastal Western", "Sahel"]
archetype_list = ["Southern", "Sahel", "Central"]
# archetype_list = ["Southern"]

In [15]:
df_list = []
for a in archetype_list:
    df_list.append(basic_scenarios(a))
df_full = pd.concat(df_list, ignore_index=True)

df_full.fillna({"smc_on": False,
                "smc_age_range": "default",
                "ivermectin": False,
                "primaquine": False}, inplace=True)

In [16]:
# Purely aesthetic: put archetype first
cols = list(df_full.columns.values)
# df_full = df_full[[cols[-1]]+cols[:-1]]
cols.remove("archetype")
cols.remove("scenario_number")
cols = ["archetype", "scenario_number"] + cols
df_full = df_full[cols]

In [17]:
df_full.to_csv("../scenario_master_list.csv", index=False)

In [18]:
df_full

,archetype,scenario_number,drug_type,screen_type,interval,itn_coverage,hs_rate,within_school_coverage,out_of_school_rate,target_age_range,smc_on,ivermectin,primaquine,smc_age_range
0,Southern,0,ASAQ,IPT,term,default,default,0.90,0.00,default,False,False,False,default
1,Southern,1,ASAQ,IPT,term,default,default,0.90,0.15,default,False,False,False,default
2,Southern,2,ASAQ,IPT,term,default,default,0.90,0.30,default,False,False,False,default
3,Southern,3,ASAQ,IPT,month,default,default,0.90,0.00,default,False,False,False,default
4,Southern,4,ASAQ,IPT,month,default,default,0.90,0.15,default,False,False,False,default
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,Central,33,DP,IPT,term,default,default,0.63,0.00,default,False,False,False,default
118,Central,34,DP,IPT,term,default,default,0.90,0.15,default,False,True,False,default
119,Central,35,DP,IPT,term,default,default,0.90,0.15,default,False,False,True,default
120,Central,36,DP,IPT,term,default,default,0.90,0.15,default,False,True,True,default


In [1]:
def basic_scenarios_DEPRECATED(archetype):
    if archetype == "Southern":
        return basic_scenarios_southern()
    else:
        raise NotImplementedError
        
        if archetype == "Southern" or archetype == "Central":
            drugs = ["ASAQ","DP"]
        elif archetype == "Eastern":
            drugs = ["ASAQ","SPAQ","DP"]
        elif archetype == "Coastal Western" or archetype == "Sahel":
            drugs = ["SPAQ","DP"]

        scenario_variations = {
            "drug_type": drugs,
            "screen_type": ["IPT", "IST"],
            "interval": ["term", "month"],
            "itn_coverage": ["default"],
            "hs_rate": ["default"],
            "within_school_coverage": [0.9],
            "out_of_school_rate": [0,0.15,0.3]
        }

        if archetype == "Sahel":
            scenario_variations["smc"] = [False, True]
        else:
            scenario_variations["smc"] = [False]

        df_combinatorial = create_df_from_combinations(scenario_variations)

        df_list = []

        high_itn_cov = pd.DataFrame({
            "drug_type": "DP",
            "screen_type": "IPT",
            "interval": "month",
            "itn_coverage": "high",
            "hs_rate": "default",
            "within_school_coverage": 0.9,
            "out_of_school_rate": 0.15,
            "smc": archetype == "Sahel"
        }, index=[0])
        df_list.append(high_itn_cov)

        low_hs_rate = pd.DataFrame({
            "drug_type": "DP",
            "screen_type": "IPT",
            "interval": "month",
            "itn_coverage": "default",
            "hs_rate": "low",
            "within_school_coverage": 0.9,
            "out_of_school_rate": 0.15,
            "smc": archetype == "Sahel"
        }, index=[0])
        df_list.append(low_hs_rate)

        all_within_school_covered = pd.DataFrame({
            "drug_type": "DP",
            "screen_type": "IPT",
            "interval": "month",
            "itn_coverage": "default",
            "hs_rate": "default",
            "within_school_coverage": 1.0,
            "out_of_school_rate": 0.15,
            "smc": archetype == "Sahel"
        }, index=[0])
        df_list.append(all_within_school_covered)

        extreme_optimist = pd.DataFrame({
            "drug_type": "DP",
            "screen_type": "IPT",
            "interval": "day",
            "itn_coverage": "default",
            "hs_rate": "default",
            "within_school_coverage": 1.0,
            "out_of_school_rate": 0,
            "smc": archetype == "Sahel"
        }, index=[0])
        df_list.append(extreme_optimist)

        no_ipt  = pd.DataFrame({
            "drug_type": "None",
            "screen_type": "None",
            "interval": "None",
            "itn_coverage": "default",
            "hs_rate": "default",
            "within_school_coverage": 0.9,
            "out_of_school_rate": 0,
            "smc": archetype == "Sahel"
        }, index=[0])
        df_list.append(no_ipt)

        itn_comparator = pd.DataFrame({
            "drug_type": "None",
            "screen_type": "None",
            "interval": "None",
            "itn_coverage": "high",
            "hs_rate": "default",
            "within_school_coverage": 0.9,
            "out_of_school_rate": 0.15,
            "smc": archetype == "Sahel"
        }, index=[0])
        df_list.append(itn_comparator)

        if archetype == "Sahel":
            no_ipt_or_smc = pd.DataFrame({
                "drug_type": "None",
                "screen_type": "None",
                "interval": "None",
                "itn_coverage": "default",
                "hs_rate": "default",
                "within_school_coverage": 0.9,
                "out_of_school_rate": 0,
                "smc": False
            }, index=[0])
            df_list.append(no_ipt_or_smc)

        df_not_combinatorial = pd.concat(df_list, ignore_index=True)

        scenario_df = pd.concat([df_combinatorial, df_not_combinatorial], ignore_index=True)
        scenario_df["scenario_number"] = range(len(scenario_df))
        scenario_df["archetype"] = archetype

        return scenario_df